# Culinária ao Redor do Mundo: Análise de Receitas

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db

Criação de Tabela

In [2]:
DROP TABLE IF EXISTS Receitas;

CREATE TABLE Receitas (
  id INTEGER NOT NULL,
  titulo VARCHAR(200),
  regiao VARCHAR(100),
  PRIMARY KEY(id)
) AS SELECT
  id, titulo, regiao
FROM CSVREAD('../data/processed/receitas.csv');


In [3]:
DROP TABLE IF EXISTS Nutrientes;

CREATE TABLE Nutrientes (
    id INTEGER NOT NULL,
    nome VARCHAR(50),
    PRIMARY KEY(id)
) AS SELECT
    id, nome
FROM CSVREAD('../data/processed/nutrientes.csv');

In [4]:
DROP TABLE IF EXISTS Compostos;

CREATE TABLE Compostos (
    id INTEGER NOT NULL,
    nome VARCHAR(300),
    PRIMARY KEY(id)
) AS SELECT
    id, nome
FROM CSVREAD('../data/processed/compostos.csv');

In [5]:
DROP TABLE IF EXISTS Ingredientes;

CREATE TABLE Ingredientes (
    id_cdb INTEGER NOT NULL,
    nome_cdb VARCHAR(300),
    id_fdb INTEGER NOT NULL,
    nome_fdb VARCHAR(300),
    grupo VARCHAR(300),
    subgrupo VARCHAR(300),
    aliases VARCHAR(300),
    PRIMARY KEY(id_cdb, id_fdb)
) AS SELECT
    id_cdb, nome_cdb, id_fdb, nome_fdb, grupo, subgrupo, aliases
FROM CSVREAD('../data/interim/ingredientes_final.csv');

In [6]:
DROP TABLE IF EXISTS ingredientes_receitas;

CREATE TABLE Ingredientes_receitas (
    id_ingrediente INTEGER NOT NULL,
    id_receita INTEGER NOT NULL,
    volume FLOAT NOT NULL,
    massa FLOAT NOT NULL,
    unidade INTEGER NOT NULL,
    PRIMARY KEY (id_ingrediente, id_receita)
) AS SELECT
    id_ingrediente, id_receita, volume, massa, unidade
FROM CSVREAD('../data/processed/ingredientes_receitas.csv');

1)Quais as estruturas bioquímicas mais frequentes em cada região?

In [7]:
SELECT regiao, nome_composto, MAX(frequencia) AS frequencia_maxima
FROM (
    SELECT r.regiao, c.nome AS nome_composto, COUNT(*) AS frequencia
    FROM Receitas r
    JOIN Ingredientes_receitas ir ON r.id = ir.id_receita
    JOIN Ingredientes i ON ir.id_ingrediente = i.id_cdb
    JOIN Compostos c ON i.id_fdb = c.id
    GROUP BY r.regiao, c.nome
) AS CompostosPorRegiao
GROUP BY regiao, nome_composto;


2)Quais regiões possuem receitas com ingredientes majoritariamente vegetais?

In [8]:
CREATE TABLE IF NOT EXISTS PorcentagemVegetaisPorReceita AS
SELECT r.id AS id_receita, r.regiao,
    COUNT(CASE WHEN i.grupo NOT IN ('Milk and milk products', 'Snack foods', 'Aquatic foods', 'Animal foods') THEN 1 END) * 100.0 / COUNT(*) AS percentual_vegetais
FROM Receitas r
JOIN Ingredientes_receitas ir ON r.id = ir.id_receita
JOIN Ingredientes i ON ir.id_ingrediente = i.id_cdb
GROUP BY r.id, r.regiao;

SELECT regiao, AVG(media_por_receita) AS media_mediavegetais_por_regiao
FROM (
    SELECT regiao, AVG(percentual_vegetais) AS media_por_receita
    FROM PorcentagemVegetaisPorReceita
    GROUP BY id_receita, regiao
) AS MediasPorReceitaPorRegiao
GROUP BY regiao;


3)Quais regiões possuem a maior média de gorduras por receita?

In [9]:
/*NÃO DÁ/*

org.h2.jdbc.JdbcSQLException:  Syntax error in SQL statement "[*]/*NÃO DÁ/*"; SQL statement

4) Qual a porcentagem de receitais que usam frutas tropicais por região?

In [10]:
SELECT regiao, AVG(freq_tropical_fruits) * 100 AS probabilidade_receita_fruta_tropical
FROM (
    SELECT r.regiao, COUNT(CASE WHEN i.subgrupo = 'Tropical fruits' THEN 1 END) * 1.0 / COUNT(DISTINCT r.id) AS freq_tropical_fruits
    FROM Receitas r
    JOIN Ingredientes_receitas ir ON r.id = ir.id_receita
    JOIN Ingredientes i ON ir.id_ingrediente = i.id_cdb
    GROUP BY r.regiao, r.id
) AS FrequenciaTropicalPorRegiao
GROUP BY regiao;
